In [17]:
import pandas as pd
import re
import pronto
import warnings
import numpy as np
import logging
import time
from pronto import Ontology
from difflib import get_close_matches
from collections import Counter
from word2number import w2n
from datetime import datetime
from urllib.parse import urlparse

warnings.filterwarnings("ignore", category=SyntaxWarning)
logging.basicConfig(level=logging.INFO)

# === Define utility ===
def normalize_string(s):
    if not isinstance(s, str):
        s = str(s)
    return s.strip().lower().replace("_", " ").replace("-", " ")

# Load CSV
input_path = r"C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\sample_run.csv"
df = pd.read_csv(input_path)
# Normalize headers
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

start = time.time()

# === Canonical Column Aliases ===
column_aliases = {
    "Cell_Type": ["celltype", "cell_type", "cell type", "Cell Type", "Cell_Type", "CellTypeSummary"],
    "Tissue": ["tissue", "tissue label", "tissue_type", "TissueType", "Tissue_id", "tissueids", "Tissue"],
    "Organ": ["organ", "organ_id", "OrganID", "Organ Id"],
    "Developmental_Stage_Summary": ["development_stage", "Developmental_Stage", "dev_stage", "stage", "devstage", "AgeGroup"],
    "Age_Summary": ["age", "Age", "age_summary"],
    "Gene_Marker_Curated": ["gene_markers", "gene_marker", "markers", "GeneMarkers", "Marker_Genes"],
    "Batch": ["batch_id", "BatchID", "batch", "batch number", "run_batch", "Batch"],
    "Platform": ["seq_platform", "platform", "seq tech", "sequencer", "sequencing_platform"],
    "Protocol": ["protocol_used", "experiment_protocol", "library_protocol", "library prep", "library_prep", "protocol", "prep method", "Library_Preparation"],
    "Ethnicity": ["ethnicity", "race", "ethnic_group", "ethnic group", "population_group", "ancestry", "Ethnicity"],
    "Disease_Summary": ["disease", "disease_status", "health_condition", "condition", "diagnosis", "DiseaseSummary"],
    "Assay": ["assay", "assay_type", "assay name", "library", "experiment type", "Assay"],
    "Donor_id": ["donor_id", "donorid", "donor id", "subject_id", "patient_id"],
    "Sample_id": ["sample_id", "sampleid", "sample id", "biosample", "biosample_id", "dataset_id"],
    "Cell_id": ["cell_id", "cellid", "cell id", "cell barcode", "barcode"],
    "Title": ["title"],
    "Sex": ["sex", "gender", "Sex"],
    "Species": ["species", "organism", "organism name", "taxonomy", "Species"],
    "Notes": ["notes", "note", "remarks", "description", "note_summary", "extra_notes"],
    "Comments": ["comments", "comment", "annotations", "curation_notes", "remarks", "comments_summary"],
    "Source": ["source", "sample_source", "origin", "material", "tissue_source", "derived_from", "Source"],
    "Sample_Type": ["sample_type", "Sample_Type", "prep_method", "sample_preparation", "source_type"],
    "File_Metadata": ["file_metadata", "filetype", "file_type", "file_format", "file_info", "metadata_file", "File_Metadata"],
    "Cell_Subtype": ["cellsubtype", "cell_subtype", "cell subtype", "Cell Subtype", "Cell_Subtype", "CellTypeSub"],
    "Technology": ["technology", "tech", "seq_tech", "sequencing_technology"],
    "Date": ["release_date", "date", "release", "public_date", "release date"],
    "Pubmed_ID": ["pubmed_id", "pubmed", "pmid"],
    "URL": ["raw_data_url", "raw_data", "url", "data_url", "accession_url"]
}

# === Column Mapping ===
def get_column(df, canonical_name):
    aliases = column_aliases.get(canonical_name, [])
    candidates = [canonical_name] + aliases
    for col in df.columns:
        col_lower = col.strip().lower().replace(" ", "_")
        if any(col_lower == candidate.strip().lower().replace(" ", "_") for candidate in candidates):
            return col
    return None

column_map = {}
for canonical_name in column_aliases:
    found_col = get_column(df, canonical_name)
    if found_col:
        column_map[canonical_name] = found_col
    else:
        logging.warning(f"⚠️ Skipping '{canonical_name}' — column not found.")

# === Helper: Get Mapped Column by Canonical Name ===
def get_mapped_col(canonical_name):
    return column_map.get(canonical_name)

# === Normalize Functions ===
def normalize_string(val):
    return str(val).strip().lower().replace("-", " ").replace("_", " ")

uncurated_terms = {}
        
# === Track uncurated terms (for reporting) ===
uncurated_terms = {
    "cell_type": set(),
    "tissue": set(),
    "organ": set(),
    "ethnicity": set(),
    "developmental_stage_summary": set(),
    "disease_summary": set(),
    "age_summary": set(),
    "sex": set(),
    "species": set(),
    "source": set(),
    "sample_type": set(),
    "file_metadata": set(),
    "assay": set(),
    "protocol": set(),
    "platform": set(),
    "batch": set(),
    "gene_marker_curated": set(),
    "notes": set(),
    "comments": set(),
    "cell_subtype": set(),
    "technology": set(),
    "date": set(),
    "pubmed_id": set(),
    "url": set(),
    "title": set()
}

print("COLUMNS IN DF:", df.columns.tolist())

# ============ CURATE AGE SECTION ============
def parse_age(raw_age):
    if pd.isna(raw_age):
        uncurated_terms["age_summary"].add("NaN")
        return pd.Series([np.nan] * 5)

    age_str = str(raw_age).strip().lower()

    # Handle common NA/unknown indicators
    if age_str in ["", "na", "n/a", "null", "none", "...", "…", "ellipsis", "unknown"]:
        uncurated_terms["age_summary"].add(age_str)
        return pd.Series([np.nan] * 5)

    manual_age_map = {
        "zygote": [0, "days", 0.0, "UBERON:0000106", "Zygote"],
        "morula": [0, "days", 0.0, "UBERON:0000107", "Embryo"],
        "blastocyst": [0, "days", 0.0, "UBERON:0000317", "Embryo"],
        "embryo": [0, "days", 0.0, "UBERON:0000068", "Embryo"],
        "fetus": [0, "days", 0.0, "UBERON:0000323", "Fetal"],
        "newborn": [0, "days", 0.0, "UO:0000033", "Neonate"],
        "p0": [0, "days", 0.0, "UO:0000033", "Neonate"],
        "p1": [1, "days", 0.003, "UO:0000033", "Neonate"],
        "p7": [7, "days", 0.02, "UO:0000033", "Infant"],
        "p14": [14, "days", 0.04, "UO:0000033", "Infant"],
        "p21": [21, "days", 0.06, "UO:0000033", "Child"],
        "infant": [0.5, "years", 0.5, "UO:0000036", "Infant"],
        "toddler": [2, "years", 2.0, "UO:0000036", "Child"],
        "child": [6, "years", 6.0, "UO:0000036", "Child"],
        "teen": [15, "years", 15.0, "UO:0000036", "Adolescent"],
        "adolescent": [16, "years", 16.0, "UO:0000036", "Adolescent"],
        "young adult": [20, "years", 20.0, "UO:0000036", "Adult"],
        "adult": [30, "years", 30.0, "UO:0000036", "Adult"],
        "mid-age": [45, "years", 45.0, "UO:0000036", "Adult"],
        "middle aged": [50, "years", 50.0, "UO:0000036", "Adult"],
        "aged": [75, "years", 75.0, "UO:0000036", "Elderly"],
        "elderly": [80, "years", 80.0, "UO:0000036", "Elderly"],
        "senior": [70, "years", 70.0, "UO:0000036", "Elderly"]
    }

    if age_str in manual_age_map:
        return pd.Series(manual_age_map[age_str])

    # Try converting words to numbers if possible
    try:
        if not any(char.isdigit() for char in age_str):
            age_str = str(w2n.word_to_num(age_str))
        else:
            words = re.findall(r'[a-zA-Z]+', age_str)
            digits = re.findall(r'\d+', age_str)
            if not digits and words:
                age_str = str(w2n.word_to_num(' '.join(words)))
    except:
        uncurated_terms["age_summary"].add(str(raw_age))
        return pd.Series([np.nan] * 5)

    # Extract value and units
    age_str = age_str.replace(" ", "")
    match = re.match(r"(\d+(?:\.\d+)?)([a-zA-Z]*)", age_str)
    if not match:
        uncurated_terms["age_summary"].add(str(raw_age))
        return pd.Series([np.nan] * 5)

    value = float(match.group(1))
    unit_key = match.group(2) if match.group(2) else 'y'
    unit_key = unit_key.lower()

    unit_map = {
        'd': ('days', 'UO:0000033'), 'day': ('days', 'UO:0000033'), 'days': ('days', 'UO:0000033'),
        'w': ('weeks', 'UO:0000034'), 'wk': ('weeks', 'UO:0000034'), 'wks': ('weeks', 'UO:0000034'),
        'week': ('weeks', 'UO:0000034'), 'weeks': ('weeks', 'UO:0000034'),
        'mo': ('months', 'UO:0000035'), 'month': ('months', 'UO:0000035'), 'months': ('months', 'UO:0000035'),
        'm': ('months', 'UO:0000035'),
        'y': ('years', 'UO:0000036'), 'yr': ('years', 'UO:0000036'), 'yrs': ('years', 'UO:0000036'),
        'year': ('years', 'UO:0000036'), 'years': ('years', 'UO:0000036')
    }

    if unit_key not in unit_map:
        uncurated_terms["age_summary"].add(str(raw_age))
        return pd.Series([value, 'unknown', np.nan, 'NA', 'Unknown'])
        
    norm_unit, uo_id = unit_map[unit_key]

    # Convert to years
    if norm_unit == 'days':
        years = round(value / 365, 2)
    elif norm_unit == 'weeks':
        years = round(value / 52.143, 2)
    elif norm_unit == 'months':
        years = round(value / 12, 2)
    else:
        years = float(value)

    # Age category
    category = (
        "Neonate" if years < 0.08 else
        "Infant" if years < 1 else
        "Child" if years < 12 else
        "Adolescent" if years < 18 else
        "Adult" if years < 65 else
        "Elderly"
    )

    return pd.Series([value, norm_unit, years, uo_id, category])

# ---- Main execution of curation ----
age_col = get_mapped_col("Age_Summary")  # Automatically get the mapped column name for Age_Summary

if age_col:
    # First, apply parsing if column exists
    try:
        df[['age_value', 'age_unit', 'age_in_years', 'age_ontology_id', 'age_category']] = df.apply(
            lambda row: parse_age(row[age_col]), axis=1, result_type='expand'
        )
    except Exception as e:
        logging.warning(f"⚠️ Error parsing age column '{age_col}': {e}")

    # Check that required columns exist
    expected_cols = ['age_value', 'age_unit', 'age_in_years', 'age_ontology_id', 'age_category']
    missing = [col for col in expected_cols if col not in df.columns]

    if missing:
        logging.warning(f"⛔ Skipping age_summary — missing columns: {missing}")
    else:
        # Define formatter
        def format_age_summary(row):
            if pd.isna(row['age_value']) or pd.isna(row['age_unit']) or pd.isna(row['age_in_years']):
                return "NA"
            try:
                value = int(float(row['age_value'])) if float(row['age_value']) == int(float(row['age_value'])) else row['age_value']
            except:
                value = row['age_value']
            return f"{value} {row['age_unit']}, {row['age_in_years']} years, {row['age_ontology_id']}, {row['age_category']}"

        # Apply formatter
        df['age_summary'] = df.apply(format_age_summary, axis=1)

        # Drop intermediate columns (but keep original 'age')
        df.drop(columns=expected_cols, inplace=True, errors='ignore')
else:
    logging.warning("⚠️ Skipping age curation — no 'Age_Summary' column mapped.")

# === Load Ontologies ===
def load_ontology(path):
    print(f"📦 Loading ontology from: {path}")
    return pronto.Ontology(path)

cl_onto = load_ontology(r"C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\cl.owl")
uberon_onto = load_ontology(r"C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\uberon.owl")
doid_onto = load_ontology(r"C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\doid.owl")
hancestro_onto = load_ontology(r"C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\hancestro.owl")

def get_term_id_and_label(ontology, name, synonyms=None, typo_corrections=None, manual_overrides=None, fallback_fuzzy=True):
    if not name or str(name).strip().lower() in ['na', 'n/a', 'null', 'none', 'nan']:
        return "NA", "NA"

    term_clean = str(name).strip().lower()

    # Manual override
    if manual_overrides and term_clean in manual_overrides:
        return manual_overrides[term_clean]

    # Apply typo corrections
    if typo_corrections:
        term_clean = typo_corrections.get(term_clean, term_clean)

    # Apply synonyms
    if synonyms:
        term_clean = synonyms.get(term_clean, term_clean)

    # Corrected loop: iterate over items, not keys
    for term_id, term in ontology.items():
        if hasattr(term, 'name') and term.name:
            if term_clean == term.name.strip().lower():
                return term_id, term.name

            # Check synonyms
            if hasattr(term, 'synonyms'):
                for syn in term.synonyms:
                    if term_clean == str(syn.description).strip().lower():
                        return term_id, term.name

    # Fallback fuzzy match
    if fallback_fuzzy:
        name_to_term = {}
        for term_id, term in ontology.items():
            if hasattr(term, 'name') and term.name:
                name_to_term[term.name.strip().lower()] = (term_id, term.name)

        from difflib import get_close_matches
        close = get_close_matches(term_clean, name_to_term.keys(), n=1, cutoff=0.85)
        if close:
            return name_to_term[close[0]]

    return "NA", "NA"
    
def map_column_with_ontology(df, column, ontology, synonyms=None, typo_corrections=None, manual_map=None):
    def normalize(val):
        val = str(val).strip().lower()
        val = re.sub(r"[^\w\s]", "", val)
        val = re.sub(r"\s+", "", val)
        return val

    ids, labels = [], []

    for val in df[column]:
        orig_val = val
        norm_val = normalize(val)

        # 1. Manual override check
        if manual_map and norm_val in manual_map:
            term_id, label = manual_map[norm_val]
        else:
            # 2. Synonym replacement
            if synonyms and norm_val in synonyms:
                val = synonyms[norm_val]

            # 3. Typo correction
            if typo_corrections and norm_val in typo_corrections:
                val = typo_corrections[norm_val]

            # 4. Use standard matcher (exact + fuzzy fallback)
            term_id, label = get_term_id_and_label(
                ontology, val,
                synonyms=synonyms,
                typo_corrections=typo_corrections,
                fallback_fuzzy=True
            )
        # Track uncurated terms
        if term_id == "NA" or not term_id:
            if column.lower() in uncurated_terms:
                uncurated_terms[column.lower()].add(str(orig_val).strip())

        ids.append(term_id if term_id else "NA")
        labels.append(label if label else "NA")

    df[f"{column}_id"] = ids
    df[f"{column}_label"] = labels

def normalize_term(value):
    if not isinstance(value, str):
        return ""
    value = value.lower().strip()
    value = re.sub(r"[-_ ]+", " ", value)
    return value
# === Disease Curation ===
manual_doid_mappings = {
    "alzheimers disease": ("DOID:10652", "Alzheimer's disease"), "dementia": ("DOID:1059", "Dementia"), "type 2 diabetes mellitus": ("DOID:9352", "Type 2 Diabetes Mellitus"),
    "alzheimers": ("DOID:10652", "Alzheimer's disease"), "systemic lupus": ("DOID:9074", "Systemic lupus erythematosus"), "sle": ("DOID:9074", "Systemic lupus erythematosus"),
    "hiv+": ("DOID:526", "HIV infectious disease"), "hiv positive": ("DOID:526", "HIV infectious disease"),
    "hiv infectious disease": ("DOID:526", "HIV infectious disease"), "hiv": ("DOID:526", "HIV infectious disease"),
    "obesity": ("DOID:9970", "Obesity"), "hepc": ("DOID:8780", "Hepatitis C"), "hepatitis b": ("DOID:2043", "Hepatitis B"),
    "hepatitis c": ("DOID:8780", "Hepatitis C"), "copd": ("DOID:3083", "Chronic obstructive pulmonary disease"),
    "chronic obstructive pulmonary disease": ("DOID:3083", "Chronic obstructive pulmonary disease"), "covid 19": ("DOID:0080600", "COVID-19"),
    "covid19": ("DOID:0080600", "COVID-19"), "covid-19": ("DOID:0080600", "COVID-19"), "COVID_19": ("DOID:0080600", "COVID-19"),"covid": ("DOID:0080600", "COVID-19"),"diabetes": ("DOID:9351", "Diabetes mellitus"),
    "type 1 diabetes": ("DOID:9744", "Type 1 diabetes mellitus"), "type i diabetes": ("DOID:9744", "Type 1 diabetes mellitus"),
    "type 2 diabetes": ("DOID:9352", "Type 2 diabetes mellitus"), "type ii diabetes": ("DOID:9352", "Type 2 diabetes mellitus"), "type 2 diabetes mellitus (t2dm)": ("DOID:9352", "Type 2 diabetes mellitus"),
    "t1d": ("DOID:9744", "Type 1 diabetes mellitus"), "t2d": ("DOID:9352", "Type 2 diabetes mellitus"), "heart failure": ("DOID:6000", "Heart failure"),
    "myocardial infarction": ("DOID:5844", "Myocardial infarction"), "coronary artery disease": ("DOID:3393", "Coronary artery disease"), "cad": ("DOID:3393", "Coronary artery disease"),
    "hypertension": ("DOID:10763", "Hypertensive disorder"), "breast cancer": ("DOID:1612", "Breast cancer"),
    "lung cancer": ("DOID:1324", "Lung cancer"), "colon cancer": ("DOID:219", "Colorectal cancer"),
    "colorectal cancer": ("DOID:219", "Colorectal cancer"), "glioblastoma": ("DOID:3068", "Glioblastoma"), "glioma": ("DOID:3060", "Glioma"), "melanoma": ("DOID:8923", "Melanoma"),
    "leukemia": ("DOID:1240", "Leukemia"),"lymphoma": ("DOID:0060058", "Lymphoma"), "multiple sclerosis": ("DOID:2377", "Multiple sclerosis"),
    "parkinson's": ("DOID:14330", "Parkinson's disease"), "parkinsons disease": ("DOID:14330", "Parkinson's disease"),
    "crohn's": ("DOID:8778", "Crohn's disease"), "ulcerative colitis": ("DOID:8577", "Ulcerative colitis"), "psoriasis": ("DOID:8893", "Psoriasis")
}

# Normalize all keys to lowercase just in case
manual_doid_mappings = {k.lower(): v for k, v in manual_doid_mappings.items()}

def normalize_disease(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '', 'none', 'nan']:
        return 'Unknown'
    return str(val).strip().lower()

def get_doid_term(name, ontology, use_fuzzy=True):
    name = name.strip().lower()

    # Exact name and synonym match
    for term in ontology.terms():
        if not term.name:
            continue
        if term.name.strip().lower() == name:
            return term.id, term.name
        if hasattr(term, 'synonyms'):
            for syn in term.synonyms:
                if name == str(syn.description).strip().lower():
                    return term.id, term.name

    # Fuzzy match fallback
    if use_fuzzy:
        all_names = {
            term.name.strip().lower(): (term.id, term.name)
            for term in ontology.terms()
            if term.name
        }

        close = get_close_matches(name, all_names.keys(), n=1, cutoff=0.88)
        if close:
            return all_names[close[0]]

    return 'Unknown', 'Unknown'

def format_disease_row(val):
    norm = normalize_disease(val)

    if norm in ['control', 'healthy', 'normal']:
        return "Healthy"
    elif norm == 'unknown':
        return "Unknown"

    if norm in manual_doid_mappings:
        doid_id, doid_label = manual_doid_mappings[norm]
        return f"{doid_id}; {doid_label}"

    doid_id, doid_label = get_doid_term(norm, doid_onto)
    if doid_id != 'Unknown':
        return f"{doid_id}; {doid_label.title()}"

    uncurated_terms["disease_summary"].add(val)

    return 'Unknown'

disease_col = get_column(df, "disease")

if disease_col:
    df[disease_col + '_original'] = df[disease_col]
    df[disease_col] = df[disease_col].apply(format_disease_row)
else:
    df['disease'] = 'Unknown'
    logging.warning("⚠️ 'disease' column not found — defaulted to 'Unknown'")

# === Developmental Stage Curation ===
def normalize_dev_stage(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '', 'nan']:
        return 'Unknown'
    return str(val).strip().lower()

manual_dev_stage_map = {
    'neonate': ('UBERON:0035820', 'neonatal stage'), 'neonatal': ('UBERON:0035820', 'neonatal stage'),
    'newborn': ('UBERON:0007220', 'newborn stage'), 'p0': ('UBERON:0007220', 'newborn stage'),
    'infant': ('UBERON:0000114', 'infant stage'), 'infancy': ('UBERON:0000114', 'infant stage'),
    'child': ('UBERON:0000106', 'child stage'), 'toddler': ('UBERON:0000112', 'toddler stage'),
    'juvenile': ('UBERON:0018241', 'juvenile stage'),
    'teenager': ('UBERON:0000116', 'pubertal stage'), 'adolescent': ('UBERON:0000116', 'pubertal stage'),
    'pubertal': ('UBERON:0000116', 'pubertal stage'), 'prepubertal': ('UBERON:0000115', 'prepubertal stage'),
    'postpubertal': ('UBERON:0000117', 'postpubertal stage'),
    'adult': ('UBERON:0000113', 'adult stage'), 'aged adult': ('UBERON:0000113', 'adult stage'),
    'early adult': ('UBERON:0018685', 'early adult stage'), 'middle aged': ('UBERON:0018684', 'middle aged stage'),
    'senescent': ('UBERON:0018686', 'senescent stage'), 'aged': ('UBERON:0010300', 'aged stage'),
    'fetus': ('UBERON:0000314', 'fetal stage'), 'fetal': ('UBERON:0000314', 'fetal stage'),
    'embryo': ('UBERON:0000068', 'embryonic stage'), 'embryonic': ('UBERON:0000068', 'embryonic stage'),
    'e12.5': ('UBERON:0000068', 'embryonic stage'),
    'postnatal': ('UBERON:0000105', 'postnatal stage'), 'perinatal': ('UBERON:0000110', 'perinatal stage'),
    'senior': ('UBERON:0010300', 'aged stage'),
    'larval': ('UBERON:0000111', 'larval stage')
}

def get_dev_stage_term(name, ontology):
    if name == 'Unknown':
        return 'Unknown', 'Unknown'

    # Manual mapping check
    if name in manual_dev_stage_map:
        return manual_dev_stage_map[name]

    # Search ontology
    for term in ontology.terms():
        if not term.name:
            continue
        if name == term.name.strip().lower():
            return term.id, term.name
        if any(name == str(s.description).strip().lower() for s in term.synonyms):
            return term.id, term.name

    # Track uncurated
    uncurated_terms["developmental_stage_summary"].add(name)
    logging.warning(f"❌ Developmental stage term not found: '{name}'")
    return 'Unknown', 'Unknown'

# === Developmental Stage Curation ===
col = get_mapped_col('Developmental_Stage_Summary')
if col:
    df[col + '_original'] = df[col]

    df['developmental_stage_norm'] = df[col].apply(normalize_dev_stage)

    df[['developmental_stage_id', 'developmental_stage_label']] = df['developmental_stage_norm'].apply(
        lambda x: pd.Series(get_dev_stage_term(x, uberon_onto))
    )

    df[col] = df['developmental_stage_id'] + "; " + df['developmental_stage_label']

    df.drop(columns=['developmental_stage_norm', 'developmental_stage_id', 'developmental_stage_label'], inplace=True)
else:
    df['developmental_stage_summary'] = 'Unknown'
    logging.warning("⚠️ 'Developmental_Stage_Summary' column not found — defaulted to 'Unknown'")

# === Ethnicity Normalization ===
def normalize_ethnicity(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '','not specified', 'nan']:
        return 'NA'
    val = str(val).strip().lower()
    normalization_dict = {
        'african': 'african american',
        'black': 'african american',
        'european': 'white',
        'caucasian': 'white',
        'latino': 'hispanic or latino',
        'hispanic': 'hispanic or latino',
        'alaska native': 'american indian or alaska native',
        'native american': 'american indian or alaska native',
        'american indian': 'american indian or alaska native',
        'pacific islander': 'native hawaiian or other pacific islander',
        'mena': 'middle eastern or north african',
        'middle eastern': 'middle eastern or north african',
        'multiple': 'mixed',
        'biracial': 'mixed',
        'asian american': 'asian',
    }
    return normalization_dict.get(val, val)

def get_ethnicity_ontology(term, ontology):
    if isinstance(term, tuple):
        return term
    if term == "NA":
        return "NA", "NA"

    term = term.strip().lower()

    manual_map = {
        'hispanic or latino': ('HANCESTRO:0014', 'Hispanic or Latino'),
        'african american': ('HANCESTRO:0004', 'African American'),
        'east asian': ('HANCESTRO:0006', 'East Asian'),
        'south asian': ('HANCESTRO:0007', 'South Asian'),
        'white': ('HANCESTRO:0005', 'European ancestry'),
        'asian': ('HANCESTRO:0008', 'Asian'),
        'american indian or alaska native': ('HANCESTRO:0013', 'American Indian or Alaska Native'),
        'native hawaiian or other pacific islander': ('HANCESTRO:0012', 'Native Hawaiian or Other Pacific Islander'),
        'mixed': ('HANCESTRO:0015', 'Mixed ancestry'),
        'middle eastern or north african': ('HANCESTRO:0011', 'Middle Eastern or North African'),
        'other': ('HANCESTRO:0016', 'Other ancestry'),
    }

    if term in manual_map:
        return manual_map[term]

    for ont_term in ontology.terms():
        if ont_term.name and ont_term.name.strip().lower() == term:
            return ont_term.id, ont_term.name
        if any(term == str(s.description).strip().lower() for s in ont_term.synonyms):
            return ont_term.id, ont_term.name

    logging.warning(f"❌ Ethnicity ontology term not found: '{term}'")
    return "NA", "NA"
    
# === Apply Ethnicity Curation ===
col = get_column(df, 'ethnicity')
if col:
    df[col + '_original'] = df[col]

    df['ethnicity_norm'] = df[col].apply(normalize_ethnicity)

    df[['ethnicity_id', 'ethnicity_label']] = df['ethnicity_norm'].apply(
        lambda x: pd.Series(get_ethnicity_ontology(x, hancestro_onto))
    )

    df[col] = df['ethnicity_id'].astype(str) + "; " + df['ethnicity_label'].astype(str)

    for raw_val in df['ethnicity_norm'].unique():
        if raw_val in ["NA", "nan", "none", "", None]:
            continue
        eid, _ = get_ethnicity_ontology(raw_val, hancestro_onto)
        if eid == 'NA':
            if "ethnicity" not in uncurated_terms:
                uncurated_terms["ethnicity"] = set()
            uncurated_terms["ethnicity"].add(raw_val)

    df.drop(columns=['ethnicity_id', 'ethnicity_label', 'ethnicity_norm'], inplace=True)

else:
    df['ethnicity'] = 'NA'
    logging.warning("⚠️ 'ethnicity' column not found — defaulted to 'NA'")
    
# === Curate Tissue ===
custom_tissue_synonyms = {
    # Blood
    "pbmc": "peripheral blood", "pbmcs": "peripheral blood",
    "peripheral blood mononuclear cell": "peripheral blood", "peripheral blood": "peripheral blood",
    "whole blood": "blood", "blood sample": "blood", "blood plasma": "blood",
    "buffycoat": "blood", "serum": "blood", "plasma": "blood",

    # Lung
    "lung tissue": "lung", "pulmonary": "lung", "lungs": "lung", "alveolar": "lung",
    "bal": "lung", "bronchoalveolar lavage": "lung", "pulmnry": "lung", "lng": "lung",
    "lung fibroblast": "lung",

    # Bone
    "bone marrow": "bone marrow", "bm": "bone marrow",

    # Skin
    "skin": "skin", "epidermis": "skin", "dermis": "skin", "keratinocyte": "skin",

    # Brain
    "brain tissue": "brain", "brain": "brain",
    "cortex": "cerebral cortex", "prefrontal cortex": "cerebral cortex",
    "hippocampus": "hippocampus", "cerebellum": "cerebellum",

    # Gut
    "colon": "colon", "rectum": "colon", "large intestine": "colon",
    "small intestine": "small intestine", "ileum": "small intestine",
    "jejunum": "small intestine", "duodenum": "small intestine",
    "intestine": "intestine", "gut": "intestine",

    # Others
    "liver": "liver", "hepatic": "liver", "kidney": "kidney", "renal": "kidney",
    "heart": "heart", "cardiac": "heart", "muscle": "muscle tissue",
    "skeletal muscle": "muscle tissue", "smooth muscle": "muscle tissue",
    "testis": "testis", "ovary": "ovary", "prostate": "prostate gland",
    "uterus": "uterus", "placenta": "placenta", "thymus": "thymus",
    "spleen": "spleen", "lymphnode": "lymph node", "tonsil": "tonsil",
    "pancreas": "pancreas", "adipose": "adipose tissue", "fat": "adipose tissue",
    "salivary gland": "salivary gland", "thyroid": "thyroid gland",
    "nasal": "nasal cavity", "nasopharynx": "nasopharynx"
}
manual_uberon_overrides = {
    "peripheralblood": ("UBERON:0013756", "Peripheral blood"), "pairoflungs": ("UBERON:0002048", "Lung"),
    "pbmc": ("UBERON:0013756", "Peripheral blood"),"pbmcs": ("UBERON:0000178", "Peripheral blood"),     "brain": ("UBERON:0000955", "brain"),
    "heart": ("UBERON:0000948", "heart"), "heart muscle": ("UBERON:0002349", "cardiac muscle tissue"), "cortex": ("UBERON:0000956", "cerebral cortex")
} 

# === Tissue Normalization ===

col = get_column(df, 'tissue')

if col:
    # 1. Preserve original column
    df[col + '_original'] = df[col]

    # 2. Normalize values
    df["tissue_norm"] = df[col].astype(str).apply(normalize_term)

    # 3. Define curation function
    def curate_tissue(val):
        if val.strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
            return pd.Series(["NA", "NA"])
        
        resolved_val = custom_tissue_synonyms.get(val, val)

        tid, label = get_term_id_and_label(
            uberon_onto,
            resolved_val,
            synonyms=custom_tissue_synonyms,
            manual_overrides=manual_uberon_overrides,
            fallback_fuzzy=True
        )
        return pd.Series([tid, label])

    # 4. Apply curation
    df[["tissue_id", "tissue_label"]] = df["tissue_norm"].apply(curate_tissue)

    # 5. Replace curated tissue column
    df[col] = df["tissue_id"] + "; " + df["tissue_label"]

    # 6. Track unmatched
    for raw_val in df["tissue_norm"].unique():
        if raw_val.strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
            continue
        tid, _ = get_term_id_and_label(
            uberon_onto,
            raw_val,
            synonyms=custom_tissue_synonyms,
            manual_overrides=manual_uberon_overrides,
            fallback_fuzzy=True
        )
        if tid == "NA":
            if "tissue" not in uncurated_terms:
                uncurated_terms["tissue"] = set()
            uncurated_terms["tissue"].add(raw_val)

    # 7. Cleanup
    df.drop(columns=["tissue_norm", "tissue_id", "tissue_label"], inplace=True)

else:
    df['tissue'] = 'NA'
    logging.warning("⚠️ 'tissue' column not found — defaulted to 'NA'")

# === Cell Type Normalization ===
def normalize_celltype(text):
    if pd.isna(text) or str(text).strip().lower() in ['na', 'n/a', 'null', 'none', '', "nan"]:
        return "NA"
    text = str(text).strip().lower()
    text = re.sub(r"[^\w\s\+]", "", text)  # Remove special characters except +
    text = re.sub(r"\s+", " ", text)
    return text

# === Raw Manual Cell Type Mapping ===
raw_manual_celltype_map = {
    "cd19+": ("CL:0000236", "CD19-positive, B cell"),
    "cd8 t cells": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd8+ t cells": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd8+ t cell": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd8t cells": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd8+": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd8tcell": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd8 t-cell": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd8positive t cells": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "tcell_cd8": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd4+ t cell": ("CL:0000624", "CD4-positive, alpha-beta T cell"),
    "cd4-positive t cell": ("CL:0000624", "CD4-positive, alpha-beta T cell"),
    "cd4 t cell": ("CL:0000624", "CD4-positive, alpha-beta T cell"),
    "cd4 t-cell": ("CL:0000624", "CD4-positive, alpha-beta T cell"),
    "cd4+ t cells": ("CL:0000624", "CD4-positive, alpha-beta T cell"),
    "cd4positive t cells": ("CL:0000624", "CD4-positive, alpha-beta T cell"),
    "nk cells": ("CL:0000623", "Natural killer cell"),
    "nk-cell": ("CL:0000623", "Natural killer cell"),
    "nk": ("CL:0000623", "Natural killer cell"),
    "natural killer cell": ("CL:0000623", "Natural killer cell"),
    "testis cells": ("CL:0000486", "testis"),
    "t cell": ("CL:0000084", "T cell"),
    "b cell": ("CL:0000236", "B cell"),
    "nk cell": ("CL:0000623", "Natural killer cell"),
    "macrophage": ("CL:0000235", "Macrophage"),
    "monocyte": ("CL:0000576", "Monocyte"),
    "m0": ("CL:0000576", "Monocyte"),
    "dendritic cell": ("CL:0000451", "Dendritic cell"),
    "neutrophil": ("CL:0000775", "Neutrophil"),
    "neu": ("CL:0000775", "Neutrophil"),
    "epithelial cell": ("CL:0000066", "Epithelial cell"),
    "endothelial cell": ("CL:0000115", "Endothelial cell"),
    "fibroblast": ("CL:0000057", "Fibroblast"),
    "basophil": ("CL:0000763", "Basophil"),
    "eosinophil": ("CL:0000771", "Eosinophil"),
    "astro": ("CL:0000127", "astrocyte"),
    "astrocyte": ("CL:0000127", "astrocyte"),
    "neuron": ("CL:0000540", "neuron"),
    "microglia": ("CL:0000129", "microglial cell"),
    "oligo": ("CL:0000128", "oligodendrocyte"),
}

# === Normalized Manual Cell Type Map ===
manual_celltype_map = {
    normalize_celltype(k): v for k, v in raw_manual_celltype_map.items()
}

# === Curate Cell Type from Ontology ===
def get_celltype_term(text, ontology):
    if not text or str(text).strip().lower() in {"na", "none", "null", "not available", "", "nan"}:
        return "NA", "NA"

    # 1. Manual map
    if text in manual_celltype_map:
        return manual_celltype_map[text]

    # 2. Exact match in ontology
    for term in ontology.terms():
        if term.name and normalize_celltype(term.name) == text:
            return term.id, term.name
        for syn in term.synonyms:
            if normalize_celltype(syn.description) == text:
                return term.id, term.name

    # 3. Fuzzy match fallback
    from difflib import get_close_matches
    all_labels = [normalize_celltype(term.name) for term in ontology.terms() if term.name]
    match = get_close_matches(text, all_labels, n=1, cutoff=0.85)
    if match:
        for term in ontology.terms():
            if term.name and normalize_celltype(term.name) == match[0]:
                return term.id, term.name

    logging.warning(f"❌ Cell type not found: '{text}'")
    return "NA", "NA"
# 1. Find the column name using column_map
cell_col = column_map.get("Cell_Type")

if cell_col:
    # Step 0: Preserve original
    df[f"{cell_col}_original"] = df[cell_col]

    # 2. Normalize values
    df["celltype_norm"] = df[cell_col].astype(str).apply(normalize_celltype)

    # 3. Define curation function
    def curate_celltype(val):
        if val.strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
            return pd.Series(["NA", "NA"])

        resolved_val = manual_celltype_map.get(val)
        if resolved_val:
            return pd.Series(resolved_val)

        # Try ontology lookup
        ct_id, label = get_celltype_term(val, cl_onto)
        return pd.Series([ct_id, label])

    # 4. Apply curation
    df[["cell_type_id", "cell_type_label"]] = df["celltype_norm"].apply(curate_celltype)

    # 5. Replace original column with ontology info
    df[cell_col] = df["cell_type_id"] + "; " + df["cell_type_label"]

    # 6. Track unmatched values
    for val in df["celltype_norm"].unique():
        if val.lower() in ["", "na", "n/a", "null", "none", "nan"]:
            continue
        ct_id, _ = get_celltype_term(val, cl_onto)
        if ct_id == "NA":
            if "cell_type" not in uncurated_terms:
                uncurated_terms["cell_type"] = set()
            uncurated_terms["cell_type"].add(val)

    # 7. Cleanup
    df.drop(columns=["celltype_norm", "cell_type_id", "cell_type_label"], inplace=True)

else:
    logging.warning("⚠️ Skipping Cell Type — no mapped column found.")

# === Cell Subtype Normalization ===
def normalize_cellsubtype(text):
    if pd.isna(text) or str(text).strip().lower() in ['na', 'n/a', 'null', 'none', '', 'nan']:
        return "NA"
    text = str(text).strip().lower()
    text = re.sub(r"[^\w\s\+]", "", text)  # Remove special characters except +
    text = re.sub(r"\s+", " ", text)
    return text

# === Manual Cell Subtype Mapping (extend as needed) ===
raw_manual_cellsubtype_map = {
    "cd8 effector memory t cell": ("CL:0000911", "CD8-positive, alpha-beta effector memory T cell"),
    "cd4 naive t cell": ("CL:0000895", "naive CD4-positive, alpha-beta T cell"),
    "cd4+ t cell": ("CL:0000624", "cd4 positive t cell"),
    "naive b cell": ("CL:0000788", "naive B cell"),
    "plasma cell": ("CL:0000786", "plasma cell"),
    "cd14+ monocyte": ("CL:0000583", "CD14-positive monocyte"),
    "gabaergic": ("CL:0000771", "GABAergic neuron"),
    "gabaergic neuron": ("CL:0000771", "GABAergic neuron"),
    "gabaergic interneuron": ("CL:0005000", "GABAergic interneuron"),
    "gabaergic cells": ("CL:0000771", "GABAergic neuron"),
    "cd8+": ("CL:0000625", "CD8-positive, alpha-beta T cell"),
    "cd4+": ("CL:0000624", "CD4-positive, alpha-beta T cell"),
    "excitatory": ("CL:0008031", "Excitatory neuron"),
    "excitatory neuron": ("CL:0008031", "Excitatory neuron"),
    "cd19+": ("CL:0000470", "CD19-positive, B cell"),
    "mo": ("CL:0000860", "Monocyte"),
    "neu": ("CL:0000775", "Neutrophil"),
    "nk": ("CL:0000623", "Natural killer cell")
    # 👆 Add more mappings here as needed
}
manual_cellsubtype_map = {
    normalize_cellsubtype(k): v for k, v in raw_manual_cellsubtype_map.items()
}

# === Curate Cell Subtype from Ontology ===
def get_cellsubtype_term(text, ontology):
    if not text or str(text).strip().lower() in {"na", "none", "null", "not available", "", "nan"}:
        return "NA", "NA"

    # 1. Manual map
    if text in manual_cellsubtype_map:
        return manual_cellsubtype_map[text]

    # 2. Exact match
    for term in ontology.terms():
        if term.name and normalize_cellsubtype(term.name) == text:
            return term.id, term.name
        for syn in term.synonyms:
            if normalize_cellsubtype(syn.description) == text:
                return term.id, term.name

    # 3. Fuzzy match
    from difflib import get_close_matches
    all_labels = [normalize_cellsubtype(term.name) for term in ontology.terms() if term.name]
    match = get_close_matches(text, all_labels, n=1, cutoff=0.85)
    if match:
        for term in ontology.terms():
            if term.name and normalize_cellsubtype(term.name) == match[0]:
                return term.id, term.name

    logging.warning(f"❌ Cell subtype not found: '{text}'")
    return "NA", "NA"

cellsub_col = get_mapped_col("Cell_Subtype")

if cellsub_col:
    # Step 0: Preserve original
    df[f"{cellsub_col}_original"] = df[cellsub_col]

    # Step 1: Normalize values
    df['cellsubtype_norm'] = df[cellsub_col].astype(str).apply(normalize_cellsubtype)

    # Step 2: Ontology mapping
    df[['cell_subtype_ontology_id', 'cell_subtype_label']] = df['cellsubtype_norm'].apply(
        lambda x: pd.Series(get_cellsubtype_term(x, cl_onto))
    )

    # Step 3: Replace original values
    df[cellsub_col] = df['cell_subtype_ontology_id'].astype(str) + "; " + df['cell_subtype_label'].astype(str)

    # Step 4: Track uncurated terms
    for val in df['cellsubtype_norm'].unique():
        if val.lower() in ["na", "none", "", "nan"]:
            continue
        ct_id, _ = get_cellsubtype_term(val, cl_onto)
        if ct_id == "NA":
            if "cell_subtype" not in uncurated_terms:
                uncurated_terms["cell_subtype"] = set()
            uncurated_terms["cell_subtype"].add(val)

    # Step 5: Clean up
    df.drop(columns=['cellsubtype_norm', 'cell_subtype_ontology_id', 'cell_subtype_label'], inplace=True)

else:
    logging.warning("⚠️ Skipping Cell Subtype — no mapped column found.")

# === Curate_Organ ===
manual_organ_overrides = {
    # Brain regions
    "brain": ("UBERON:0000955", "brain"),
    "cerebrum": ("UBERON:0000956", "cerebrum"),
    "cerebellum": ("UBERON:0002037", "cerebellum"),
    "cortex": ("UBERON:0000956", "cerebral cortex"),
    "prefrontal cortex": ("UBERON:0016529", "prefrontal cortex"),
    "hippocampus": ("UBERON:0001954", "hippocampus"),
    "amygdala": ("UBERON:0001876", "amygdala"),

    # Lung and respiratory
    "lung": ("UBERON:0002048", "lung"),
    "lungs": ("UBERON:0002048", "lung"),
    "pairoflungs": ("UBERON:0002048", "lung"),
    "pulmonary": ("UBERON:0002048", "lung"),
    "bronchi": ("UBERON:0002185", "bronchus"),
    "alveoli": ("UBERON:0002295", "alveolus"),
    "trachea": ("UBERON:0003126", "trachea"),
    "nasopharynx": ("UBERON:0001728", "nasopharynx"),

    # Liver, kidney, and GI
    "liver": ("UBERON:0002107", "liver"),
    "hepatic": ("UBERON:0002107", "liver"),
    "kidney": ("UBERON:0002113", "kidney"),
    "renal": ("UBERON:0002113", "kidney"),
    "intestine": ("UBERON:0000160", "intestine"),
    "small intestine": ("UBERON:0002108", "small intestine"),
    "large intestine": ("UBERON:0000059", "large intestine"),
    "colon": ("UBERON:0001155", "colon"),
    "duodenum": ("UBERON:0002114", "duodenum"),
    "jejunum": ("UBERON:0002115", "jejunum"),
    "ileum": ("UBERON:0002116", "ileum"),
    "stomach": ("UBERON:0000945", "stomach"),
    "esophagus": ("UBERON:0001043", "esophagus"),

    # Cardiovascular
    "heart": ("UBERON:0000948", "heart"),
    "cardiac": ("UBERON:0000948", "heart"),
    "aorta": ("UBERON:0000946", "aorta"),
    "ventricle": ("UBERON:0002071", "cardiac ventricle"),

    # Reproductive
    "testis": ("UBERON:0000473", "testis"),
    "ovary": ("UBERON:0000992", "ovary"),
    "uterus": ("UBERON:0000995", "uterus"),
    "placenta": ("UBERON:0001987", "placenta"),
    "prostate": ("UBERON:0002367", "prostate gland"),

    # Lymphatic & immune
    "spleen": ("UBERON:0002106", "spleen"),
    "thymus": ("UBERON:0002370", "thymus"),
    "lymph node": ("UBERON:0000029", "lymph node"),
    "tonsil": ("UBERON:0000341", "tonsil"),

    # Endocrine
    "thyroid": ("UBERON:0002046", "thyroid gland"),
    "adrenal gland": ("UBERON:0002369", "adrenal gland"),
    "pancreas": ("UBERON:0001264", "pancreas"),
    "pituitary": ("UBERON:0000007", "pituitary gland"),

    # Skin & Musculoskeletal
    "skin": ("UBERON:0002097", "skin"),
    "epidermis": ("UBERON:0001003", "epidermis"),
    "muscle": ("UBERON:0002385", "muscle tissue"),
    "skeletal muscle": ("UBERON:0001134", "skeletal muscle tissue"),

    # Blood and derivatives
    "blood": ("UBERON:0000178", "blood"),
    "pbmc": ("UBERON:0013756", "peripheral blood"),
    "pbmcs": ("UBERON:0013756", "peripheral blood"),
    "bone marrow": ("UBERON:0002371", "bone marrow"),

    # Sense organs
    "eye": ("UBERON:0000970", "eye"),
    "retina": ("UBERON:0000966", "retina"),
    "ear": ("UBERON:0001690", "ear"),
    "cochlea": ("UBERON:0001756", "cochlea"),

    # Other
    "salivary gland": ("UBERON:0001836", "salivary gland"),
    "nasal cavity": ("UBERON:0001707", "nasal cavity")
}

organ_col = get_mapped_col("Organ")

if organ_col:
    # Step 0: Preserve original
    df[f"{organ_col}_original"] = df[organ_col]

    # Step 1: Normalize input
    df["organ_norm"] = df[organ_col].fillna("").astype(str).str.strip().str.lower()

    # Step 2: Define curation logic
    def curate_organ(val):
        if val in ["", "na", "n/a", "null", "none"]:
            return pd.Series(["NA", "NA"])

        resolved_val = custom_tissue_synonyms.get(val, val)

        tid, label = get_term_id_and_label(
            uberon_onto,
            resolved_val,
            synonyms=custom_tissue_synonyms,
            manual_overrides=manual_uberon_overrides,
            fallback_fuzzy=True
        )
        return pd.Series([tid, label])

    # Step 3: Apply curation
    df[["organ_id", "organ_label"]] = df["organ_norm"].apply(curate_organ)

    # Step 4: Replace original column with curated info
    df[organ_col] = df["organ_id"].astype(str) + "; " + df["organ_label"].astype(str)

    # Step 5: Track uncurated terms
    for raw_val in df["organ_norm"].unique():
        if raw_val in ["", "na", "n/a", "null", "none"]:
            continue
        tid, _ = get_term_id_and_label(
            uberon_onto,
            raw_val,
            synonyms=custom_tissue_synonyms,
            manual_overrides=manual_uberon_overrides,
            fallback_fuzzy=True
        )
        if tid == "NA":
            if "organ" not in uncurated_terms:
                uncurated_terms["organ"] = set()
            uncurated_terms["organ"].add(raw_val)

    # Step 6: Cleanup
    df.drop(columns=["organ_norm", "organ_id", "organ_label"], inplace=True)

else:
    logging.warning("⚠️ Skipping Organ curation — no suitable organ column found.")

# === Batch Curation ===
def curate_batch(val):
    if pd.isna(val) or str(val).strip().lower() in ["na", "n/a", "null", "none", "nan", ""]:
        return "NA"
    val = str(val).strip()
    clean_val = re.sub(r'[^a-zA-Z0-9]', '', val)
    return f"Batch_{clean_val.upper()}" if clean_val else "NA"

batch_col = get_mapped_col("Batch")

if batch_col:
    # Step 0: Preserve original column
    df[f"{batch_col}_original"] = df[batch_col]

    # Step 1: Apply batch curation
    df[batch_col] = df[batch_col].apply(curate_batch)
else:
    logging.warning("⚠️ Skipping Batch — no suitable 'batch' column found.")

# === Sex Curation ===
def normalize_sex(value):
    value = str(value).strip().lower()
    if value in ["m", "male"]:
        return "Male"
    elif value in ["f", "female"]:
        return "Female"
    elif value == "intersex":
        return "Other"
    elif value in ["nonbinary", "na", "nill", "no", "", "other"]:
        return "Unknown"
    else:
        return "Unknown"  # tracking happens below

sex_col = get_mapped_col("Sex")

if sex_col:
    # Step 0: Preserve original column
    df[f"{sex_col}_original"] = df[sex_col]

    # Step 1: Normalize values
    df[sex_col] = df[sex_col].apply(normalize_sex)

    # Step 2: Track uncurated values
    for val in df[f"{sex_col}_original"].astype(str).str.strip().str.lower().unique():
        if val not in ["m", "male", "f", "female", "intersex", "nonbinary", "na", "nill", "no", "", "other"]:
            if "sex" not in uncurated_terms:
                uncurated_terms["sex"] = set()
            uncurated_terms["sex"].add(val)
else:
    logging.warning("⚠️ Skipping Sex — no mapped column found.")

# === Species Curation ===
def normalize_species(value):
    value = str(value).strip().lower()
    if value in ["human", "hs", "homo sapiens", "H. sapiens"]:
        return "Homo sapiens"
    elif value in ["mouse", "mus musculus", "wt", "balb/c", "ko", "c57bl/6"]:
        return "Mus musculus"
    elif value in ["rat", "rattus norvegicus"]:
        return "Rattus norvegicus"
    else:
        return "Unknown"  # Tracking done below

species_col = get_mapped_col("Species")

if species_col:
    # Step 0: Save original
    df[f"{species_col}_original"] = df[species_col]

    # Step 1: Normalize
    df[species_col] = df[species_col].apply(normalize_species)

    # Step 2: Track uncurated values
    for val in df[f"{species_col}_original"].astype(str).str.strip().str.lower().unique():
        if val not in [
            "human", "hs", "homo sapiens",
            "mouse", "mus musculus", "wt", "balb/c", "ko", "c57bl/6",
            "rat", "rattus norvegicus"
        ]:
            if "species" not in uncurated_terms:
                uncurated_terms["species"] = set()
            uncurated_terms["species"].add(val)
else:
    logging.warning("⚠️ Skipping Species — no mapped column found.")

# === Load HGNC Mappings ===
hgnc_df = pd.read_csv(r"C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\hgnc_complete_set.txt", sep="\t", dtype=str)
hgnc_df = hgnc_df[['HGNC_ID', 'Approved_symbol', 'Ensembl_gene_ID']].dropna()

# === Build Lookup Dictionaries ===
ensg_to_hgnc = hgnc_df.set_index('Ensembl_gene_ID')[['HGNC_ID', 'Approved_symbol']].to_dict(orient='index')
hgnc_to_ensembl = hgnc_df.set_index('HGNC_ID')[['Approved_symbol', 'Ensembl_gene_ID']].to_dict(orient='index')
symbol_to_info = hgnc_df.set_index('Approved_symbol')[['HGNC_ID', 'Ensembl_gene_ID']].to_dict(orient='index')
alias_to_symbol = {}  # Optional: Fill with alias mappings if available

# === Gene_Marker_Curated with Synonym Support ===
unmatched_genes = set()

def curate_gene_marker(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '', 'not available']:
        return [("NA", "NA", "NA")]

    val = str(val).strip().upper()
    genes = [g.strip() for g in re.split(r'[;,]+', val) if g.strip()]
    curated = []

    for gene in genes:
        original_gene = gene

        if gene.startswith("ENSG"):
            info = ensg_to_hgnc.get(gene)
            if info:
                curated.append((info.get('Approved_symbol', "NA"), info.get('HGNC_ID', "NA"), gene))
            else:
                curated.append(("NA", "NA", gene))
                unmatched_genes.add(gene)

        elif gene.startswith("HGNC:"):
            info = hgnc_to_ensembl.get(gene)
            if info:
                curated.append((info.get('Approved_symbol', "NA"), gene, info.get('Ensembl_gene_ID', "NA")))
            else:
                curated.append(("NA", gene, "NA"))
                unmatched_genes.add(gene)

        elif gene in symbol_to_info:
            info = symbol_to_info[gene]
            curated.append((gene, info.get('HGNC_ID', "NA"), info.get('Ensembl_gene_ID', "NA")))

        elif gene in alias_to_symbol:
            main_symbol = alias_to_symbol[gene]
            info = symbol_to_info.get(main_symbol)
            if info:
                curated.append((main_symbol, info.get('HGNC_ID', "NA"), info.get('Ensembl_gene_ID', "NA")))
            else:
                curated.append((main_symbol, "NA", "NA"))
                unmatched_genes.add(gene)

        else:
            curated.append((gene, "NA", "NA"))
            unmatched_genes.add(gene)

    return curated

def join_curated_genes(curated_list):
    return " | ".join([f"{sym}; {hgnc}; {ensg}" for sym, hgnc, ensg in curated_list])

# === Gene Marker Curation ===
if "Gene_Marker_Curated" in column_map:
    curated_col = column_map.get("Gene_Marker_Curated", "Gene_Marker_Curated")

    # Step 0: Save original values
    df[curated_col + "_original"] = df[curated_col]

    # Step 1: Curate gene symbols
    df[curated_col] = df[curated_col].apply(lambda x: join_curated_genes(curate_gene_marker(x)))

    # Step 2: Track unmatched gene symbols
    if unmatched_genes:
        if "Gene_Marker_Curated" not in uncurated_terms:
            uncurated_terms["Gene_Marker_Curated"] = set()
        uncurated_terms["Gene_Marker_Curated"].update(unmatched_genes)

else:
    logging.warning("⚠️ Skipping Gene Marker Curation — no mapped 'Gene_Marker_Curated' column found.")
        
# === curate_donor_id ===
def format_donor_id(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'nan', 'null', 'none', '']:
        return 'Unknown'
    return str(val).strip()

# Handle 'Donor_id' column whether it exists or not
col = get_column(df, 'donor_id')
if col:
    df[col + '_original'] = df[col]
    df[col] = df[col].apply(format_donor_id)
else:
    df['donor_id'] = 'Unknown'
    
# === curate_sample_id ===
def format_sample_id(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '']:
        return "Unknown"
    return str(val).strip()

col = get_column(df, 'dataset_id')
if col:
    df[col + '_original'] = df[col]
    df[col] = df[col].apply(format_sample_id)
else:
    df['dataset_id'] = 'Unknown'

# === curate_cell_id ===
def cell_id_normalize(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '']:
        return "Unknown"
    return str(val).strip()

col = get_column(df, 'cell_id')
if col:
    df[col + '_original'] = df[col]
    df[col] = df[col].apply(cell_id_normalize)
else:
    df['cell_id'] = 'Unknown'

# === curate_notes ===
def keyword_match(val, keywords):
    return any(re.search(rf"\b{k}\b", val) for k in keywords)

def simplify_notes(notes):
    if pd.isna(notes) or str(notes).strip().lower() in ['na', 'n/a', 'null', 'none', '', 'NA']:
        return "NA"

    val = str(notes).strip().lower()

    mapping = {
        "Check consent forms": ["check consent form", "check consent"],
        "Unclear": ["unclear", "unknown", "bad", "low quality", "poor quality"],
        "OK": ["no issues", "ok", "all good", "no problem", "looks fine", "clean"],
        "Flag_review": ["check", "pending", "review", "verify", "unclear", "check image", "confirm"],
        "QC Passed": ["qc passed", "good", "verified"],
        "Single-Nucleus": ["nucleus"]
    }

    for label, keywords in mapping.items():
        if keyword_match(val, keywords):
            return label

    return "Custom-note"

col = get_column(df, 'Notes')  # or whichever column you're targeting

if col:
    df[col + '_original'] = df[col]
    df[col] = df[col].apply(simplify_notes)
else:
    logging.warning("⚠️ 'Notes' column not found — skipping Notes curation.")

# === curate_comments ===
def keyword_match(val, keywords):
    """Returns True if any keyword is found as a word or substring in val."""
    return any(re.search(rf"\b{k}\b", val) for k in keywords)

def simplify_comments(comments):
    if pd.isna(comments) or str(comments).strip().lower() in ['na', 'n/a', 'none', 'null', '', 'NA']:
        return "NA"

    val = str(comments).strip().lower()

    mapping = {
        "Smoker": ["smoke", "smoker", "smoking"],
        "Diabetic": ["diabetes", "diabetic", "t2dm", "type 2 diabetes", "type ii diabetes"],
        "Check consent forms": ["check consent", "verify consent"],
        "Alcohol use": ["alcohol", "drink", "drinking", "alcoholic"],
        "Cancer history": ["cancer", "tumor", "carcinoma", "malignancy"],
        "OK": ["no comment", "ok", "no issues", "fine"],
        "Flag_review": ["needs review", "uncertain", "check", "unsure", "verify", "investigate"]
    }

    for label, keywords in mapping.items():
        if keyword_match(val, keywords):
            return label

    return "Unknown"

col = get_column(df, 'Comments')  # Or whatever column you're working on

if col:
    df[col + '_original'] = df[col]  # Optional: keep original
    df[col] = df[col].apply(simplify_comments)
else:
    logging.warning("⚠️ Skipping 'Comments' — column not found.")

# === Curate Assay ===
manual_assay_synonyms = {}

# Assay keyword mapping
assay_keywords = {
        "RNA seq": ["rnaseq", "rna-seq", "rna seq", "bulk rna seq", "bulk rnaseq", "rna sequencing", "transcriptome sequencing", "whole transcriptome", "total rna-seq"],
        "CEL Seq2": ["celseq2", "cel-seq2"],
        "sci RNA seq": ["scirnaseq", "sci-rna-seq", "sci rna seq"],
        "Microwell seq": ["microwell", "microwellseq", "microwell-seq"],
        "scRNA seq": ["scrnaseq", "scrna-seq", "sc-rna seq", "smartseq", "10x", "chromium", "dropseq", "seqwell", "indrops", "truseq", "scrbseq", "splitseq", "singlecell", "single cell rna", "single cell transcriptome"],
        "snRNA seq": ["snrna", "singlenucleus", "nucleus", "single nucleus", "sn-rna", "snrna-seq"],
        "CITE seq": ["citeseq", "adt", "hashtag", "cite-seq", "cite seq", "rna+protein", "rna + surface protein", "scrna + protein", "scrna + citeseq", "scRNA + surface protein"],
        "ATAC seq": ["atac seq", "atacseq", "chromatin accessibility", "open chromatin profiling"],
        "scATAC seq": ["scatac", "scatacseq", "sc-atac", "sc-atac-seq", "singlecell atac", "single cell atac"],
        "snATAC seq": ["snatac", "sn-atac", "sn atac", "single nucleus atac"],
        "Multiome": ["multiome", "rna+atac", "multiomic", "multi-omic", "combined rna atac", "joint profiling", "rna and atac", "multi-modal"],
        "MARS seq": ["marsseq", "mars-seq"],
        "BD Rhapsody": ["bd rhapsody", "rhapsody"],
        "ChIP seq": ["chipseq", "chip-seq", "chromatin immunoprecipitation"],
        "SNARE seq": ["snareseq", "snare-seq"],
        "Unknown": ["", "other"]
}

unmatched_assay_values = set()

# Helper function
def normalize_string(val):
    if pd.isna(val):
        return ""
    return str(val).strip().lower().replace("-", "").replace(" ", "")

def curate_assay(val):
    if pd.isna(val) or val.strip().lower() in ['na', 'n/a', 'null', 'none', '']:
        return "Unknown"

    val_norm = normalize_string(val)
    resolved = manual_assay_synonyms.get(val_norm, val_norm)

    for label, keywords in assay_keywords.items():
        if any(k in resolved for k in keywords):
            return label

    unmatched_assay_values.add(val)
    return "Unknown"

# Apply
col = get_column(df, 'assay')

if col:
    # Step 0: Preserve original
    df[col + '_original'] = df[col]

    # Step 1: Normalize input
    df[col] = df[col].astype(str).str.strip().str.lower()

    # Step 2: Apply curation
    df[col] = df[col].apply(curate_assay)

    # Step 3: Rename curated column to standardized name
    df.rename(columns={col: 'assay'}, inplace=True)

else:
    logging.warning("⚠️ Skipping Assay — no suitable 'assay' column found.")

# Step 4: Log unmatched values
if unmatched_assay_values:
    if "assay" not in uncurated_terms:
        uncurated_terms["assay"] = set()
    uncurated_terms["assay"].update(unmatched_assay_values)
    logging.warning(f"⚠️ Unmatched assay values: {sorted(unmatched_assay_values)}")

# === Source Curation ===
source_keywords = {
        "PBMC": ["pbmc", "peripheral blood mononuclear cells", "mononuclear cells"],
        "Whole Blood": ["blood", "whole blood", "venous blood", "buffy coat", "plasma"],
        "Bone Marrow": ["bone marrow"],
        "iPSC-derived neurons": ["ipsc", "induced pluripotent stem cells", "pluripotent stem cell", "ipsc-derived neurons", "ipsc neuron", "neurons derived from ipsc", "ipsc derived neurons", "neuron (ipsc)", "reprogrammed", "ipsc-derived", "induced pluripotent", "ipsc line"],
        "Embryonic Stem Cell": ["esc", "embryonic stem cell", "es cell"],
        "Cell Line": ["cell line", "hela", "hek293", "immortalized", "cell culture", "hek", "293t", "jurkat"],
        "Primary Cells": ["primary cells", "primary culture"],
        "Biopsy": ["biopsy", "biopsied tissue", "needle biopsy", "tissue biopsy", "tumor", "solid tissue"],
        "Tissue": ["tissue", "brain", "lung", "heart", "skin", "liver", "brain tissue", "kidney"],
        "Tumor": ["tumor", "carcinoma", "cancer", "adenocarcinoma", "neoplasm", "malignant"],
        "Healthy": ["healthy", "wildtype", "normal", "control"],
        "Treated": ["treated", "stimulated", "drug treated", "perturbed"],
        "Organoid": ["organoid", "3d culture", "organoids"],
        "Sorted Cells": ["facs", "macs", "sorted", "facs sorted", "magnetic beads"],
        "Nucleus": ["nucleus", "nuclei", "snrna", "single nucleus", "single nuclei"],
        "Other": ["mixed", "pooled sample", "unspecified", "various", "na"],
        "In vitro": ["in vitro", "cultured", "culture", "cell culture"],
        "Xenograft": ["xenograft", "pdx"],
        "Urine": ["urine", "urinary", "urinary tract", "urine sample"],
        "CSF": ["csf", "cerebrospinal"],
        "Saliva": ["saliva", "oral", "buccal"],
        "Nasal": ["nasal", "nasopharyngeal", "nose", "np"],
        "Sputum": ["sputum", "phlegm"],
        "Unknown": []
}
manual_source_synonyms = {}
unmatched_source_values = set()

def curate_source(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '', 'nan']:
        return "Unknown"
    val_norm = normalize_string(val)
    resolved = manual_source_synonyms.get(val_norm, val_norm)
    for label, keywords in source_keywords.items():
        if any(k in resolved for k in keywords):
            return label
    unmatched_source_values.add(val)
    return "Unknown"

# === Source Curation ===
col = get_column(df, 'source')

if col:
    # Step 0: Backup original values
    df[col + '_original'] = df[col]

    # Step 1: Normalize raw input
    df[col] = df[col].astype(str).str.strip().str.lower()

    # Step 2: Apply source curation
    df[col] = df[col].apply(curate_source)

    # Step 3: Rename curated column to standard name
    df.rename(columns={col: 'source'}, inplace=True)

else:
    logging.warning("⚠️ Skipping Source — no suitable 'source' column found.")

# Step 4: Log unmatched values
if unmatched_source_values:
    if "source" not in uncurated_terms:
        uncurated_terms["source"] = set()
    uncurated_terms["source"].update(unmatched_source_values)
    logging.warning(f"⚠️ Unmatched source values: {sorted(unmatched_source_values)}")

# === Protocol Curation ===
protocol_keywords = {
        "Smart seq2": ["smartseq", "smartseq2", "smartseq v2", "smartseqv2", "smartseq2v3", "smart-seq", "smartseq2 v2.1", "smartseq2.0", "smart-seq2", "ss2", "ssv2"],
        "10x chromium": ["10x", "10xgenomics", "10x genomics", "chromium", "10x chromium", "chromiumv2", "chromiumv3", "chromiumv3.1", "10x chromium v2", "10x chromium v3", "10x chromium v3.1", "chromiumv2.0", "chromiumv3.0", "10x v3", "10xv2", "10xv3"],
        "Drop seq": ["dropseq", "drop-seq", "dropseq-based"],
        "Seq-Well": ["seqwell", "seq-well", "seqwell platform"],
        "SPLiT seq": ["splitseq", "split-seq", "splitseq-based"],
        "CITE seq": ["citeseq", "cite-seq", "cite seq", "rna+adt", "rna+protein", "rna + surface protein"],
        "TruSeq": ["truseq", "truseq-stranded", "truseq rna", "truseq stranded mrna", "truseq total rna", "illumina truseq"],
        "fluidigm": ["fluidigm", "fluidigm c1", "c1", "fluidigm-c1", "fluidigm c1 single cell autoprep", "c1 platform"],
        "SCRB seq": ["scrbseq", "scrb-seq", "scrb"],
        "Ribo-depletion": ["ribodepletion", "ribo-depleted", "ribo depletion", "ribo depleted", "ribozero", "ribo-zero", "ribo zero"],
        "SMARTer": ["smarter", "smarter-seq", "smarter ultra", "clontech smarter", "smarter pico"],
        "NEBNext": ["nebnext", "neb next", "nebnext ultra", "nebnext stranded", "nebnext rna"],
        "CEL Seq2": ["celseq", "cel-seq", "celseq2", "cel-seq2"],
        "MARS seq": ["marsseq", "mars-seq"],
        "BD Rhapsody": ["bd rhapsody", "rhapsody"],
        "Microwell seq": ["microwell", "microwellseq", "microwell-seq"],
        "InDrops": ["indrops", "in-drops", "in drops"],
        "Tagmentation": ["tagmentation", "tagment", "tagmented"],
        "PolyA selection": ["polya", "poly a", "poly-a", "polyadenylation", "polya selection"],
        "Nugen Ovation": ["nugen", "nugen ovation"],
        "Other": ["nanodrop", "qubit", "tapestation", "bioanalyzer"] 
}
unmatched_protocol_values = set()

def curate_protocol(val):
    if pd.isna(val) or str(val).strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
        return "Unknown"
    val_norm = normalize_string(val)
    for label, keywords in protocol_keywords.items():
        if any(k in val_norm for k in keywords):
            return label
    unmatched_protocol_values.add(val)
    return "Unknown"

# === Protocol Curation ===
col = get_column(df, 'protocol')

if col:
    # Step 0: Backup original
    df[col + '_original'] = df[col]

    # Step 1: Normalize
    df[col] = df[col].astype(str).str.strip().str.lower()

    # Step 2: Curate
    df[col] = df[col].apply(curate_protocol)

    # Step 3: Rename to standard
    df.rename(columns={col: 'protocol'}, inplace=True)

    # Step 4: Track uncurated terms
    if unmatched_protocol_values:
        if "protocol" not in uncurated_terms:
            uncurated_terms["protocol"] = set()
        uncurated_terms["protocol"].update(unmatched_protocol_values)

else:
    logging.warning("⚠️ Skipping Protocol — no suitable 'protocol' column found.")
    
# === Sample_Type Curation ===
sample_type_keywords = {
        "Fresh": ["fresh", "fresh sample", "fresh tissue"],
        "Frozen": ["frozen", "flash frozen", "snap frozen"],
        "FFPE": ["ffpe", "formalin-fixed paraffin-embedded", "formalin fixed", "paraffin embedded", "formalin", "formalin-fixed", "paraffin"],
        "Bulk": ["bulk", "bulk sample", "bulk rna", "whole sample"],
        "Single Cell": ["single cell", "sc", "sc sample", "scRNA", "single-cell"],
        "Single Nucleus": ["nucleus", "nuclei", "sn", "snrna sample", "single nucleus", "singlenucleus"],
        "Unknown": ["na", "unspecified", "unknown", "not reported", ""],
        "Biopsy": ["biopsy", "needle biopsy", "tissue biopsy"],
        "Scrape": ["scrape", "skin scrape", "swab"],
        "Sorted cells": ["sorted", "sorted cells", "facs sorted", "cell sorting"]
}
unmatched_sample_type_values = set()

def curate_sample_type(val):
    if pd.isna(val) or str(val).strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
        return "Unknown"
    val_norm = normalize_string(val)
    for label, keywords in sample_type_keywords.items():
        if any(k in val_norm for k in keywords):
            return label
    unmatched_sample_type_values.add(val)
    return "Unknown"

col = get_column(df, 'sample_type')

if col:
    # Step 0: Backup original
    df[col + '_original'] = df[col]

    # Step 1: Normalize
    df[col] = df[col].astype(str).str.strip().str.lower()

    # Step 2: Curate
    df['sample_type'] = df[col].apply(curate_sample_type)

    # Step 3: Track unmatched terms
    if unmatched_sample_type_values:
        if "sample_type" not in uncurated_terms:
            uncurated_terms["sample_type"] = set()
        uncurated_terms["sample_type"].update(unmatched_sample_type_values)

else:
    logging.warning("⚠️ Skipping Sample Type — no suitable 'sample_type' column found.")

# === Sequencing_Platform Curation ===
sequencing_platform_keywords = {
        "Illumina HiSeq": ["hiseq", "hiseq2000", "hiseq2500", "hiseq3000", "hiseq4000"],
        "Illumina": ["illumina", "10x genomics", "dropseq", "smartseq", "10x", "drop-seq", "smart-seq"],
        "Illumina NovaSeq": ["novaseq", "novaseq6000", "novaseq x", "novaseqx"],
        "PacBio": ["pacbio", "smrt", "hifi"],
        "ONT MinION": ["minion", "ont", "ont minion", "nanopore minion"],
        "ONT PromethION": ["promethion", "ont promethion"],
        "Ion Torrent PGM": ["iontorrent", "ion torrent", "ion pgm", "pgm"],
        "Unknown": []
}
unmatched_sequencing_platform_values = set()

def curate_sequencing_platform(val):
    if pd.isna(val) or str(val).strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
        return "Unknown"
    val_norm = normalize_string(val)
    for label, keywords in sequencing_platform_keywords.items():
        if any(k in val_norm for k in keywords):
            return label
    unmatched_sequencing_platform_values.add(val)
    return "Unknown"

col = get_column(df, 'platform')

if col:
    df['platform_original'] = df[col]
    df[col] = df[col].astype(str).str.strip().str.lower()
    df['platform'] = df[col].apply(curate_sequencing_platform)  # overwrite same name
    if unmatched_sequencing_platform_values:
        uncurated_terms["platform"] = unmatched_sequencing_platform_values
else:
    logging.warning("⚠️ Skipping Platform — no suitable 'platform' column found.")
    
# === Technology Curation ===
technology_keywords = {
        "ATAC seq": ["atac seq", "atacseq", "chromatin accessibility", "sc-atac", "bulk atac", "atac sequencing"],
        "snRNA seq": ["snrna", "snrna-seq", "single nucleus rna seq", "sn-rna", "single nucleus", "sn rna-seq"],
        "scRNA seq": ["scrna", "scrnaseq", "sc-rna", "sc-rna seq", "singlecell rna seq", "single cell rna seq", "sc rna-seq", "sc rna seq", "single-cell transcriptomics", 
                      "10x", "10x genomics", "10x chromium", "chromium", "sc transcriptomics", "sc transcriptome"],
        "Multiome": ["multiome", "multi omic", "multi-omic", "multiomic", "rna+atac", "rna and atac", "joint profiling", "combined rna atac"],
        "Bulk RNA seq": ["rnaseq", "rna-seq", "rna seq", "bulk rna seq", "bulk rnaseq", "bulkseq", "bulk", "bulk rna", "rna sequencing"],
        "Unknown": ["", "other"]
}
unmatched_technology_values = set()

def curate_technology(val):
    if pd.isna(val) or str(val).strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
        return "Unknown"
    val_norm = normalize_string(val)
    for label, keywords in technology_keywords.items():
        if any(k in val_norm for k in keywords):
            return label
    unmatched_technology_values.add(val)
    return "Unknown"

# === Apply Curation ===
col = get_column(df, 'technology')

if col:
    # Step 0: Backup original
    df[col + '_original'] = df[col]

    # Step 1: Normalize input
    df[col] = df[col].astype(str).str.strip().str.lower()

    # Step 2: Apply curation
    df['technology'] = df[col].apply(curate_technology)

    # Step 3: Track unmatched terms
    if unmatched_technology_values:
        if "technology" not in uncurated_terms:
            uncurated_terms["technology"] = set()
        uncurated_terms["technology"].update(unmatched_technology_values)

else:
    logging.warning("⚠️ Skipping technology — no suitable 'technology' column found.")
    
# === File_Metadata Curation ===
file_metadata_keywords = {
    "Bcl": ["bcl", "basecall"],
    "Fastq": ["fastq", "fq", ".fastq", ".fq"],
    "Sra": ["sra", ".sra", "sequence read archive"],
    "Bam": ["bam", ".bam"],
    "Cram": ["cram", ".cram"],
    "Sam": ["sam", ".sam"],
    "Vcf": ["vcf", ".vcf", "variant", "genotype"],
    "Mtx": ["mtx", ".mtx", "sparse matrix"],
    "Hdf5": ["hdf5", ".h5", ".hdf5", "10x h5", "cellranger output"],
    "Csv": ["csv", ".csv", "comma-separated"],
    "Tsv": ["tsv", ".tsv", "tab-separated"],
    "Txt": ["txt", ".txt", "plain text"],
    "Loom": ["loom", ".loom"],
    "AnnData": ["anndata", ".h5ad", "h5ad", "scanpy"],
    "Seurat": ["seurat", ".rds", "rds", "seurat object", "r object"],
    "CellRanger": ["cellranger", "outs", "filtered_feature_bc_matrix", "molecule_info"],
    "STARsolo": ["starsolo", "star", "solo", "star output"],
    "Salmon/Alevin": ["salmon", "alevin", "quant.sf", "tximport"],
    "Gtf": ["gtf", ".gtf", "gene annotation"],
    "Gff": ["gff", ".gff", "gff3"],
    "Fasta": ["fasta", ".fasta", ".fa", "reference genome"],
    "Index": ["index", ".idx", "bowtie", "hisat", "salmon index"],
    "Zip": [".zip", "zip file", "compressed"],
    "Tar": [".tar", "tar.gz", ".tgz"],
    "Gz": [".gz", "gzip", "gzipped", "fastq.gz", "bam.gz"],
    "Metadata": ["metadata", "sample info", "sample_sheet", "samplesheet.csv"],
    "Json": ["json", ".json", "config"],
    "Xml": ["xml", ".xml"],
    "Excel": ["xls", "xlsx", ".xls", ".xlsx", "excel"],
    "Pdf": ["pdf", ".pdf", "report"],
    "Html": ["html", ".html", "multiqc"],
    "Log": ["log", ".log", "run log"],
    "QcReport": ["fastqc", "qc", "multiqc", "quality control"],
    "Unknown": []   
}
unmatched_file_values = set()

def curate_file_metadata(val):
    if pd.isna(val) or str(val).strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
        return "Unknown"
    val_norm = str(val).strip().lower()
    for label, keywords in file_metadata_keywords.items():
        if any(k in val_norm for k in keywords):
            return label
    return "Unknown"

file_meta_col = column_map.get("File_Metadata")

if file_meta_col and file_meta_col in df.columns:
    # Step 0: Backup original
    df[file_meta_col + '_original'] = df[file_meta_col]

    # Step 1: Apply curation
    df[file_meta_col] = df[file_meta_col].apply(curate_file_metadata)

    # Step 2: Track unmatched values if available
    if 'unmatched_file_values' in globals() and unmatched_file_values:
        if "File_Metadata" not in uncurated_terms:
            uncurated_terms["File_Metadata"] = set()
        uncurated_terms["File_Metadata"].update(unmatched_file_values)

else:
    logging.warning("⚠️ Skipping File_Metadata — no mapped column found.")
    
# === Date Curation ===
from datetime import datetime

# Try parsing with multiple known formats
KNOWN_DATE_FORMATS = [
    "%d/%m/%Y",  # 20/05/2021
    "%Y-%m-%d",  # 2021-05-20
    "%m/%d/%Y",  # 05/20/2021
    "%Y/%m/%d",  # 2021/05/20
    "%d-%m-%Y",  # 20-05-2021
    "%m-%d-%Y",  # 05-20-2021
    "%d %b %Y",  # 20 May 2021
    "%b %d, %Y", # May 20, 2021
]

def normalize_release_date(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '', 'nan', 'unknown']:
        return "Unknown"

    val = str(val).strip()
    
    for fmt in KNOWN_DATE_FORMATS:
        try:
            parsed = datetime.strptime(val, fmt)
            return parsed.strftime("%Y-%m-%d")  # Always return ISO format
        except ValueError:
            continue

    return "Invalid"  # If all formats fail

# Apply to dataframe
release_col = get_column(df, 'Date')
if release_col:
    df[release_col + '_original'] = df[release_col]  # Preserve original
    df[release_col] = df[release_col].apply(normalize_release_date)
else:
    logging.warning("⚠️ Skipping Date — no suitable 'Date' column found.")


# === Title Curation ===
title_replacements = {
    r"\bscRNA[-\s]?seq\b": "single-cell RNA sequencing",
    r"\bPBMC\b": "peripheral blood mononuclear cells",
    r"^Dataset from\s+": "",  # Remove prefix
}

unmatched_title_values = set()

def curate_title(val):
    if pd.isna(val) or str(val).strip().lower() in ["", "na", "n/a", "null", "none", "nan"]:
        return "Unknown", val
    
    original_val = val.strip()
    curated = original_val

    # Apply all regex substitutions
    for pattern, replacement in title_replacements.items():
        curated = re.sub(pattern, replacement, curated, flags=re.IGNORECASE)

    # Sentence case
    if curated:
        curated = curated[0].upper() + curated[1:]

    # Track unmatched/modified
    if curated != original_val:
        unmatched_title_values.add(original_val)

    return curated, original_val

# Apply only if column found
col = get_column(df, 'title')
if col:
    df['title_original'] = df[col]
    df['title_curated'] = df[col].apply(curate_title)

    if unmatched_title_values:
        uncurated_terms.setdefault("title", set()).update(unmatched_title_values)
else:
    logging.warning("⚠️ Skipping Title — no suitable 'title' column found.")

# === URL Curation ===
import pandas as pd
import re
import logging
from urllib.parse import urlparse

def extract_ids_from_text(text):
    ids = set()

    # GEO (NCBI or ArrayExpress)
    ids.update(re.findall(r'\b(GSE\d+|E-GEOD-\d+)\b', text, re.IGNORECASE))

    # SRA
    ids.update(re.findall(r'\b(SRR\d+|SRX\d+|SRS\d+|SRP\d+)\b', text, re.IGNORECASE))

    # BioProject / BioSample
    ids.update(re.findall(r'\b(PRJ[NED][A-Z]?\d+|SAM[NES]?\d+)\b', text, re.IGNORECASE))

    # CellxGene
    if "cellxgene" in text:
        cxg_match = re.search(r'/collections/([^/?#]+)', text)
        if cxg_match:
            ids.add(f"cellxgene:{cxg_match.group(1)}")

    # Human Cell Atlas (UUID-like project IDs)
    if "humancellatlas" in text:
        hca_match = re.search(r'/projects/([a-f0-9\-]{10,})', text)
        if hca_match:
            ids.add(f"hca:{hca_match.group(1)}")

    return ";".join(sorted(ids)) if ids else "Unmatched"

def curate_url_or_id(val):
    if pd.isna(val) or str(val).strip().lower() in ['na', 'n/a', 'null', 'none', '', 'nan', 'unknown']:
        return "NA; NA; NA"

    val = str(val).strip()
    url_like = bool(re.match(r'^(https?|ftp)://', val)) or "www." in val

    # Normalize if starts with www
    if val.startswith("www."):
        val = "https://" + val

    if url_like:
        parsed = urlparse(val)
        host = parsed.netloc.lower()

        # Classify host/platform
        if "ncbi.nlm.nih.gov" in host:
            host_type = "NCBI"
        elif "ebi.ac.uk" in host:
            host_type = "EBI"
        elif "cellxgene" in host:
            host_type = "CellxGene"
        elif "humancellatlas" in host:
            host_type = "HCA"
        else:
            host_type = "Other"

        ids = extract_ids_from_text(val)
        norm_url = re.sub(r'^http:', 'https:', val)  # normalize to https

        return f"{norm_url}; {host_type}; {ids}"

    else:
        # Handle if input is only an accession ID
        ids = extract_ids_from_text(val)
        if ids != "Unmatched":
            # Guess host type from prefix
            if val.upper().startswith("GSE") or val.upper().startswith("E-GEOD"):
                host_type = "NCBI"
            elif val.upper().startswith("SR") or val.upper().startswith("PRJ"):
                host_type = "SRA"
            elif val.lower().startswith("cellxgene"):
                host_type = "CellxGene"
            elif val.lower().startswith("hca"):
                host_type = "HCA"
            else:
                host_type = "Other"
            return f"ID_only:{val}; {host_type}; {ids}"
        else:
            return f"{val}; Unknown; FLAG: Review"

url_col = get_column(df, 'URL')  # Replace this with your actual column detection method

if url_col:
    df[url_col + '_original'] = df[url_col]  # preserve raw input
    df[url_col] = df[url_col].apply(curate_url_or_id)
else:
    logging.warning("⚠️ Skipping URL curation — no suitable 'URL' column found.")

# === Final Diagnostics ===
print("FINAL SHAPE:", df.shape)

# ======= Final_Stage_Select: curated + original side-by-side =======

final_cols_to_keep = []

# Build side-by-side curated + original columns
for canonical, curated_col in column_map.items():
    if curated_col in df.columns:
        final_cols_to_keep.append(curated_col)
        original_col = curated_col + "_original"
        if original_col in df.columns:
            final_cols_to_keep.append(original_col)

# Identify other columns not part of column_map (e.g., 'age_summary', etc.)
remaining_cols = [col for col in df.columns if col not in final_cols_to_keep]
final_cols_to_keep.extend(remaining_cols)

# Log any expected but missing canonical columns
expected_canonicals = list(column_aliases.keys())
missing = [name for name in expected_canonicals if name not in column_map]
if missing:
    logging.warning(f"⚠️ Missing expected canonical columns: {missing}")

# Final selection
df = df[final_cols_to_keep]

print("✅ Columns kept (ordered curated + original pairs):", final_cols_to_keep)
print("✅ Shape after column filtering:", df.shape)

# === Save File ===
output_path = r"C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\Harmonized_metadata.csv"
df.to_csv(output_path, index=False)
print("✅ Final harmonized metadata saved to:", output_path)

# === Save uncurated terms to file ===
all_uncurated = []

for col_name, terms in uncurated_terms.items():
    # Convert each term to string before sorting to avoid TypeError
    for term in sorted(map(str, terms)):
        all_uncurated.append({
            "Column": col_name,
            "Uncurated_Term": term
        })

if all_uncurated:
    uncurated_df = pd.DataFrame(all_uncurated)
    uncurated_df.to_csv("uncurated_terms.csv", index=False)
    print("🧾 Saved uncurated terms to 'uncurated_terms.csv'")

print("\n✅ Script completed in", round(time.time() - start, 2), "seconds")

    

COLUMNS IN DF: ['dataset_id', 'title', 'platform', 'technology', 'file_metadata', 'raw_data_url']
📦 Loading ontology from: C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\cl.owl


C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000116' at 0x000001E1C728D440>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000116' at 0x000001E1C728E840>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000116' at 0x000001E1CD9B7560>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://www.w3.org/2000/01/rdf-schema#}seeAlso' at 0x000001E1CD9B7F10>
  self._extract_object_p

📦 Loading ontology from: C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\uberon.owl


C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000112' at 0x000001E1D1599AD0>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000112' at 0x000001E1D159A2F0>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000112' at 0x000001E1D159A5C0>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000116' at 0x000001E1D159B5B0>
  self._extract_object_pro

📦 Loading ontology from: C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\doid.owl


C:\Users\kiran\miniconda3\Lib\site-packages\pronto\ontology.py:290: SyntaxWarning: could not find axiom source: 'IAO:0000115'
  cls(self).parse_from(_handle)  # type: ignore
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\ontology.py:290: SyntaxWarning: could not find axiom source: 'http://purl.org/dc/elements/1.1/description'
  cls(self).parse_from(_handle)  # type: ignore
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\ontology.py:290: SyntaxWarning: could not find axiom source: 'http://purl.org/dc/elements/1.1/title'
  cls(self).parse_from(_handle)  # type: ignore
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\ontology.py:290: SyntaxWarning: could not find axiom source: 'http://purl.org/dc/terms/license'
  cls(self).parse_from(_handle)  # type: ignore
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\ontology.py:290: SyntaxWarning: could not find axiom source: 'http://www.geneontology.org/formats/oboInOwl#hasDbXref'
  cls(self).parse_from(_handle)  # type: ignore
C:\Users

📦 Loading ontology from: C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\ontologies\hancestro.owl


C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000111' at 0x000001E1C329F970>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000112' at 0x000001E1C329F790>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000112' at 0x000001E1C329D620>
  self._extract_object_property(prop, curies)
C:\Users\kiran\miniconda3\Lib\site-packages\pronto\parsers\rdfxml.py:114: SyntaxWarning: unknown element in `owl:ObjectProperty`: <Element '{http://purl.obolibrary.org/obo/}IAO_0000112' at 0x000001E1C329CFE0>
  self._extract_object_pro

FINAL SHAPE: (7, 19)
✅ Columns kept (ordered curated + original pairs): ['platform', 'platform_original', 'dataset_id', 'dataset_id_original', 'title', 'title_original', 'file_metadata', 'file_metadata_original', 'technology', 'technology_original', 'raw_data_url', 'raw_data_url_original', 'disease', 'developmental_stage_summary', 'ethnicity', 'tissue', 'donor_id', 'cell_id', 'title_curated']
✅ Shape after column filtering: (7, 19)
✅ Final harmonized metadata saved to: C:\Users\kiran\OneDrive\Desktop\Jesus_Synlico\Harmonized_metadata.csv
🧾 Saved uncurated terms to 'uncurated_terms.csv'

✅ Script completed in 35.86 seconds
